In [1]:
#Importation des librairies utilisées
import unicodedata 
import time
import pandas as pd
import numpy as np
import random
import nltk
import collections
import itertools
import csv
import warnings
import pickle
import scipy

import keras.models as km
import keras.layers as kl


In [2]:
DATA_DIR = "/Users/bguillouet/Insa/TP_Insa/data/cdiscount_NLP/text_generation/"

chars = pickle.load(open(DATA_DIR+"char_reduit_list","rb")) + ["start","end"]
X = np.load(DATA_DIR+"Coque_description_char_reduit_197.npy")

In [3]:
int_to_char = {i:c for i,c in enumerate(chars)}
char_to_int = {c:i for i,c in int_to_char.items()}
I_START = char_to_int["start"]
I_END = char_to_int["end"]

In [4]:
SIZE_VOCAB = len(chars)
LENGTH_SEQUENCE = len(X[0])+1
N_X = len(X)
print("Number of sentences : %d, Vocab size : %d, Sequence Length : %d" %(N_X, SIZE_VOCAB, LENGTH_SEQUENCE))

Number of sentences : 1761637, Vocab size : 90, Sequence Length : 198


In [5]:
def encode_input_output_sequence(x, length_sequence, size_vocab, char_to_int_dic, i_start, i_end):
    x_vec = np.zeros((length_sequence, size_vocab))
    y_vec = np.zeros((length_sequence, size_vocab))
    x_vec[0][i_start] = 1
    y_vec[-1][i_end] = 1
    for ix,c in enumerate(x):
        c_int = char_to_int_dic[c]
        x_vec[ix+1][c_int]=1
        y_vec[ix][c_int]=1
    return x_vec, y_vec


def decode_sequence(x, int_to_char_dic):
    seq = []
    for i in np.where(x)[1]:
        seq.append(int_to_char_dic[i])
    return "".join(seq)


In [6]:
N_vec=128000
X_vec = np.zeros((N_vec, LENGTH_SEQUENCE, SIZE_VOCAB))
X_vec[:,0,I_START]=1
Y_vec = np.zeros((N_vec, LENGTH_SEQUENCE, SIZE_VOCAB))
Y_vec[:,-1,I_END]=1
for ix, x in enumerate(X[:N_vec]):
    if ix%100 == 0:
        print("%d/%d"%(ix,N_vec), end="\r")
    for ic,c in enumerate(x):
        c_int = char_to_int[c]
        X_vec[ix][ic+1][c_int]=1
        Y_vec[ix][ic][c_int]=1


In [7]:
nb_hidden = 32
epochs = 20
batch_size=128

model = km.Sequential()
model.add(kl.LSTM(nb_hidden, input_shape=(None, SIZE_VOCAB), return_sequences=True))
model.add(kl.TimeDistributed(kl.Dense(SIZE_VOCAB)))
model.add(kl.Activation('softmax'))
model.compile(loss="categorical_crossentropy", optimizer="rmsprop")
model.fit(X_vec, Y_vec, epochs=epochs, batch_size=batch_size)

Epoch 1/20
128000/128000 [==============================] - 864s - loss: 2.5434   
Epoch 2/20
128000/128000 [==============================] - 865s - loss: 1.1466   
Epoch 3/20
128000/128000 [==============================] - 863s - loss: 0.6351   
Epoch 4/20
128000/128000 [==============================] - 862s - loss: 0.4348   
Epoch 5/20
128000/128000 [==============================] - 862s - loss: 0.3507   
Epoch 6/20
128000/128000 [==============================] - 865s - loss: 0.3152   
Epoch 7/20
128000/128000 [==============================] - 866s - loss: 0.2940   
Epoch 8/20
128000/128000 [==============================] - 867s - loss: 0.2837   
Epoch 9/20
128000/128000 [==============================] - 898s - loss: 0.2749   
Epoch 10/20
128000/128000 [==============================] - 878s - loss: 0.2690   
Epoch 11/20
128000/128000 [==============================] - 877s - loss: 0.2645   
Epoch 12/20
128000/128000 [==============================] - 878s - loss: 0.2609   
E

In [8]:
i_test = 0
print(decode_sequence(X_vec[i_test], int_to_char))
Y = model.predict(X_vec[i_test].reshape(1,LENGTH_SEQUENCE, SIZE_VOCAB))


startPour apple iphone 4 : coque bumper silicone blanc - Cet étui en silicone rigide protège et habille votre APPLE iPhone 4. Parfaitement adapté, il permet l'accès à toutes les fo… Voir la présentation


In [9]:
seq=[]
for i in Y[0].argmax(1):
    seq.append(int_to_char[i])
"".join(seq)

'Sour sppeeccoeene c - coque peteart-on g q  -landhd *httmce lln -onegene panene fiauere tn corelee cotre -naaa -naone C  Careeie  e   cne pee coepossen eeacees à toutes les peu Voir la présentationend'

In [123]:
X_pred = np.zeros((1, LENGTH_SEQUENCE, SIZE_VOCAB))
print("step 0")
X_pred[0,0,I_START] = 1
X_pred_str = decode_sequence(X_pred[0], int_to_char)
print(X_pred_str)

#print("step 1")
#l = "C"
#i_l = char_to_int[l]
#X_pred[0,1,i_l] = 1
#X_pred_str = decode_sequence(X_pred[0], int_to_char)
#print(X_pred_str)

for i in range(197):
    i=i
    ix = np.argmax(model.predict(X_pred[:,:i+1,:])[0][-1,:])
    X_pred[0,i+1,ix] = 1
    X_pred_str = decode_sequence(X_pred[0], int_to_char)
    print(X_pred_str, end="\r")

step 0
start


In [134]:
def my_normalize(v):
    norm = sum(v)
    return v/norm*0.99

X_pred = np.zeros((1, LENGTH_SEQUENCE, SIZE_VOCAB))
print("step 0")
X_pred[0,0,I_START] = 1
X_pred_str = decode_sequence(X_pred[0], int_to_char)
print(X_pred_str)

#print("step 1")
#l = "C"
#i_l = char_to_int[l]
#X_pred[0,1,i_l] = 1
#X_pred_str = decode_sequence(X_pred[0], int_to_char)
#print(X_pred_str)

for i in range(197):
    i=i
    predict_step = my_normalize(model.predict(X_pred[:,:i+1,:])[0][-1,:])
    ix = np.argmax(np.random.multinomial(1,predict_step, size=1))
    X_pred[0,i+1,ix] = 1
    X_pred_str = decode_sequence(X_pred[0], int_to_char)
    print(X_pred_str, end="\r")

step 0
start
